# Procesando coordenadas de los cortaderos

Procesando datos .kml que los obtenemos al exportar los datos de mymaps.

In [1]:
import json

from bs4 import BeautifulSoup
import numpy as np

In [2]:
def read4json(file_path):
    "Read configuration data form json file and return a list"
    with open(file_path, 'r') as data_file:
        data = json.load(data_file)
    return data

def save2json(file_path, data):
    "Save configuration data to a json file"
    with open(file_path, 'w') as data_file:
        json.dump(data, data_file, indent=4)

In [3]:
censo_path = '../datos/crudos/censo2010cba.json'
kml_path = '../datos/crudos/Ladrilleros.kml'
data_path = '../datos/cortaderos_new.geojson'

In [4]:
# diccionario usados para obtener los valores del censo

acceso_a_gas = {
    'H14_01': 'Gas de red',
    'H14_02': 'Gas a granel',
    'H14_03': 'Gas en tubo',
    'H14_04': 'Gas en garrafa',
    'H14_05': 'Electricidad',
    'H14_06': 'Leña o carbón',
    'H14_07': 'Otro'
}

acceso_a_agua = {
    'H12_01': 'Red pública',
    'H12_02': 'Perforación con bomba a motor',
    'H12_03': 'Perforación con bomba manual',
    'H12_04': 'Pozo',
    'H12_05': 'Transporte por cisterna',
    'H12_06': 'Agua de lluvia, río, canal, arroyo o acequia'
}

H12 = ['H12_01', 'H12_02', 'H12_03', 'H12_04', 'H12_05', 'H12_06']
H14 = ['H14_01', 'H14_02', 'H14_03', 'H14_04', 'H14_05', 'H14_06', 'H14_07']

In [5]:
data = open(kml_path).read()
bs = BeautifulSoup(data, "lxml")

In [6]:
bs.find('coordinates').text

'-63.9524943,-31.3270035,0.0'

In [7]:
censo = read4json(censo_path)
len_censo = len(censo[0]['features'])

In [8]:
todas_cortaderas = []

for coordenadas in bs.find_all('coordinates'):
    # Coordenadas de la cortadera
    lat, long, z = coordenadas.text.split(',')
    cortadera_pos = np.array([float(lat), float(long)])
    
    # calculo ladistancia de la cortadera a todos los puntos del censo
    pos = np.zeros((len_censo, 2))
    for i in range(len_censo):
        pos[i, 0] = censo[0]['features'][i]['properties']['coordinates'][0]
        pos[i, 1] = censo[0]['features'][i]['properties']['coordinates'][1]

    diff = (pos - cortadera_pos)**2
    suma = diff[:,0] + diff[:,1]
    minimo = suma.argmin()
    
    # obtengo los datos del agua del punto mas cercano del censo
    agua_dic = {}
    for key in H12:
        value = censo[0]['features'][minimo]['properties'][key]
        new_key = acceso_a_agua[key]
        agua_dic[new_key] = value

    # obtengo los datos del gas del punto mas cercano del censo
    gas_dic = {}    
    for key in H14:
        value = censo[0]['features'][minimo]['properties']['H14_01']
        new_key = acceso_a_gas[key]
        gas_dic[new_key] = value
        
    # densidaddemografica
    densidad = censo[0]['features'][minimo]['properties']['P_TOTAL']
    
    # nuevo valor para el json de las cortaderas
    entry = {
            "geometry": {
                "coordinates": list(cortadera_pos),
                "type": "Point"
            },
            "type": "Feature",
            "properties": {
                "name": "Cortadero",
                "acceso_gas": gas_dic,
                "acceso_agua": agua_dic,
                "densidad": densidad
            }
        }

    todas_cortaderas.append(entry)

In [9]:
geojson = {
    "type": "FeatureCollection",
    "features": todas_cortaderas
    }

save2json('../datos/cortaderos_new.geojson', geojson)